In [49]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import math as math
from scipy.optimize import minimize
import scipy.stats as ss
from scipy.signal import find_peaks 
from matplotlib import rc
from scipy.optimize import fsolve

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 20

In [50]:
#open files to read
z = 4
a = 8
daughter_file = open("/home/jeppe/devel/resources/geant4/geant4-v11.0.2-MOD-install/share/Geant4-11.0.2/data/RadioactiveDecay5.6Old/z"+str(z)+".a"+str(a),"r")
hasNext = True
while(hasNext):
    hasNext = daughter_file.readline()
    #print(hasNext)
daughter_file.close()

In [51]:
#retrieve all nominal levels in width_file and their associated sublevels and BRs
width_file = open("/home/jeppe/devel/projects/geant4/sim/DecayFileGenerator/WidthFiles/z"+str(z)+"WIDTH.a"+str(a),"r")
nominalLevels = []
sublevelArray = []
BRarray = []
currentSubLevels = []
currentBRs = []

moveOn = True
i = 0
while(moveOn):
    moveOn = width_file.readline()
    if(moveOn[:12] == 'NOMINALLEVEL'):
        nominalLevels.append(float(moveOn[12:]))
        if(i != 0):
            sublevelArray.append(currentSubLevels)
            BRarray.append(currentBRs)
            currentBRs = []
            currentSubLevels = []
    else:
        if (moveOn==''): 
            continue
        ENandBR = moveOn.split("\n")[0].split("\t")
        currentBRs.append(float(ENandBR[2]))
        currentSubLevels.append(float(ENandBR[1]))

    i += 1

sublevelArray.append(currentSubLevels)
BRarray.append(currentBRs)

In [52]:
sublevelArray[1]

[7050.0,
 7050.0,
 7051.0,
 7051.0,
 7052.0,
 7053.0,
 7053.0,
 7054.0,
 7054.0,
 7055.0,
 7056.0,
 7056.0,
 7057.0,
 7057.0,
 7058.0,
 7059.0,
 7059.0,
 7060.0,
 7060.0,
 7061.0,
 7062.0,
 7062.0,
 7063.0,
 7063.0,
 7064.0,
 7065.0,
 7065.0,
 7066.0,
 7066.0,
 7067.0,
 7068.0,
 7068.0,
 7069.0,
 7069.0,
 7070.0,
 7071.0,
 7071.0,
 7072.0,
 7072.0,
 7073.0,
 7074.0,
 7074.0,
 7075.0,
 7075.0,
 7076.0,
 7077.0,
 7077.0,
 7078.0,
 7078.0,
 7079.0,
 7080.0,
 7080.0,
 7081.0,
 7081.0,
 7082.0,
 7083.0,
 7083.0,
 7084.0,
 7084.0,
 7085.0,
 7086.0,
 7086.0,
 7087.0,
 7087.0,
 7088.0,
 7089.0,
 7089.0,
 7090.0,
 7090.0,
 7091.0,
 7092.0,
 7092.0,
 7093.0,
 7093.0,
 7094.0,
 7095.0,
 7095.0,
 7096.0,
 7096.0,
 7097.0,
 7098.0,
 7098.0,
 7099.0,
 7099.0,
 7100.0,
 7101.0,
 7101.0,
 7102.0,
 7102.0,
 7103.0,
 7104.0,
 7104.0,
 7105.0,
 7105.0,
 7106.0,
 7107.0,
 7107.0,
 7108.0,
 7108.0,
 7109.0,
 7110.0,
 7110.0,
 7111.0,
 7111.0,
 7112.0,
 7113.0,
 7113.0,
 7114.0,
 7114.0,
 7115.0,
 7116.0,
 

In [53]:
#now, in a modified daughterfile, create new decay definitions for all the sublevels (equal to the decay definitions in the original daughterfile, with different q-vals)
daughter_file = open("/home/jeppe/devel/resources/geant4/geant4-v11.0.2-MOD-install/share/Geant4-11.0.2/data/RadioactiveDecay5.6Old/z"+str(z)+".a"+str(a),"r")
modified_daughter_file = open("ModifiedDecayFiles/z"+str(z)+".a"+str(a),"w")

hasNext = True

notRead = True
while(hasNext):
    if(notRead): 
        hasNext = daughter_file.readline()
    else:
        hasNext = decayMode
    notRead = True
    if(hasNext == ''): continue

    modified_daughter_file.write(hasNext)

    if(hasNext[0] == 'P'):
        parentString = hasNext
        splitted = hasNext.split()
        currentNominal = float(splitted[1])

        for i in range(len(nominalLevels)):
            if(abs(currentNominal - nominalLevels[i]) < 0.01):
                #found a match. All the next lines until either end of file or the next P-level are the decay modes for the level.
                moveOn = True
                decayModes = []
                while(moveOn):
                    decayMode = daughter_file.readline()

                    if(decayMode == ''):
                        moveOn = False
                        hasNext = False
                        #write all decaymodes for the sublevels aswell.
                        #replace nominalLevel in parentString with sublevel energy:
                        newParentString = parentString.replace(splitted[1],"Hej")
                        #write all decaymodes for the sublevels aswell.
                        for subLevel in sublevelArray[i]:
                            #replace nominalLevel in parentString with sublevel energy:
                            newParentString = parentString.replace(' ' + splitted[1] + ' ',' ' +str(subLevel)+ ' ')
                            modified_daughter_file.write(newParentString)
                            #also write all decay modes
                            for decay in decayModes:
                                #Q-value of decay must be modified accordingly
                                if(len(decay) > 72):
                                    #decay contains Q-value.
                                    Qvalue = decay.split()[-1]
                                    decay = decay.replace(' ' + Qvalue, ' ' + str(float(Qvalue) + (subLevel-currentNominal)))
                                modified_daughter_file.write(decay)
                    if(decayMode != '' and decayMode[0] == 'P'):
                        moveOn = False
                        notRead = False
                        #write all decaymodes for the sublevels aswell.
                        for subLevel in sublevelArray[i]:
                            #replace nominalLevel in parentString with sublevel energy:
                            newParentString = parentString.replace(' ' + splitted[1] + ' ',' ' +str(subLevel)+ ' ')
                            modified_daughter_file.write(newParentString)
                            #also write all decay modes
                            for decay in decayModes:
                                #Q-value of decay must be modified accordingly
                                if(len(decay) > 72):
                                    #decay contains Q-value.
                                    Qvalue = decay.split()[-1]
                                    decay = decay.replace(' ' + Qvalue, ' ' + str(float(Qvalue) + (subLevel-currentNominal)))
                                modified_daughter_file.write(decay)
                    else:
                        modified_daughter_file.write(decayMode)
                        decayModes.append(decayMode)
daughter_file.close()
modified_daughter_file.close()